## Exploration d'un dataset "Effectif de patient atteints de cancers pris en charge par l'assurance maladie en ile de france"

### I. Récupération des données et importation dans DuckDB

In [1]:
import requests
import pandas as pd
import duckdb

url = "https://data.ameli.fr/api/explore/v2.1/catalog/datasets/effectifs/records"

params = {
    "limit": 100,
    "where": "patho_niv1 = 'Cancers' AND region = '11'"
}

response = requests.get(url, params=params)
response.raise_for_status()
data = response.json()

# Les résultats sont dans "results"
records = data.get("results", [])
df = pd.DataFrame(records)

print(df.head())


# Importation dans DuckDB 
con = duckdb.connect("ameli_cancers.duckdb")

# Création d'une table et insertion du DataFrame
con.register("df_view", df)
con.execute("CREATE TABLE IF NOT EXISTS cancers_idf AS SELECT * FROM df_view")

# Supprimer la colonne 'tri' si elle existe
con.execute("ALTER TABLE cancers_idf DROP COLUMN IF EXISTS tri")

# Vérification : lire les 5 premières lignes depuis DuckDB, table cancers_idf
result = con.execute("SELECT * FROM cancers_idf LIMIT 5").fetchdf()
print("\nDonnées lues depuis DuckDB :\n")
print(result)



  annee patho_niv1 patho_niv2 patho_niv3          top cla_age_5 sexe region  \
0  2020    Cancers       None       None  CAN_CAT_CAT     00-04    1     11   
1  2020    Cancers       None       None  CAN_CAT_CAT     00-04    1     11   
2  2020    Cancers       None       None  CAN_CAT_CAT     00-04    1     11   
3  2020    Cancers       None       None  CAN_CAT_CAT     00-04    1     11   
4  2020    Cancers       None       None  CAN_CAT_CAT     00-04    1     11   

  dept  ntop   npop   prev niveau_prioritaire libelle_classe_age libelle_sexe  \
0   75    50  51830  0.093                [1]       de 0 à 4 ans       hommes   
1   77    40  44980  0.091                [1]       de 0 à 4 ans       hommes   
2   78    40  45740  0.083                [1]       de 0 à 4 ans       hommes   
3   91    60  43420  0.127                [1]       de 0 à 4 ans       hommes   
4   92    40  52030  0.071                [1]       de 0 à 4 ans       hommes   

   tri  
0  1.0  
1  1.0  
2  1.0  
3 

II. Exploration statistique interactive avec la librairIe ydata_profiling

In [2]:
from ydata_profiling import ProfileReport 
from IPython.display import HTML

# Connexion à la base DuckDB: ameli_cancers
con = duckdb.connect("ameli_cancers.duckdb")

# Requête SQL pour récupérer toutes les données de la table cancer_idf
df = con.execute("SELECT * FROM cancers_idf").fetchdf()

# Générer un rapport statistique descriptif 
profile = ProfileReport(df, title="Rapport descriptif - Cancers IDF", explorative=True)

# Sauvegarder en HTML
profile.to_file("rapport_cancers_idf.html")
print("Rapport généré avec succès : rapport_cancers_idf.html")

# Crée un lien pour télécharger le rapport HTML
HTML('<a href="rapport_cancers_idf.html" download>Télécharger le Rapport Cancers IDF</a>')

C:\Users\slima\anaconda3\envs\testech_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Export report to file: 100%|██████████| 1/1 [00:00<00:00, 64.87it/s]

Rapport généré avec succès : rapport_cancers_idf.html
